In [114]:
from pyspark import SparkContext
import re
import random
import numpy as np
from numpy.linalg import norm
import hashlib
from datetime import datetime

In [2]:
'''
    hdfs dfs -mkdir -p assign2
    
'''

'\n    hdfs dfs -mkdir -p assign2\n    \n'

In [3]:
sc = SparkContext(master='local', appName="Assignment2_E2")

In [4]:
# Finding similar Users

In [5]:
ratings_data = sc.textFile("assign2/ratings.csv")
header = ratings_data.first() #extract header
ratings_data = ratings_data.filter(lambda line: line != header)   #filter out header

In [6]:
data = ratings_data.map(lambda line: re.split(',', line.lower()))

In [7]:
data.take(5)

[['1', '1', '4.0', '964982703'],
 ['1', '3', '4.0', '964981247'],
 ['1', '6', '4.0', '964982224'],
 ['1', '47', '5.0', '964983815'],
 ['1', '50', '5.0', '964982931']]

In [83]:
# Get ratings by user
by_users = data.map(lambda elem: (elem[0],(elem[1], elem[2]))).groupByKey().mapValues(list)

In [24]:
num_users = by_users.count()
num_users

610

In [26]:
# Get ratings by user
by_movie = data.map(lambda elem: (elem[1],(elem[0], elem[2]))).groupByKey().mapValues(list)
num_movies = by_movie.count()
num_movies

9724

In [100]:
def get_matrix(by_movie, num_users):
    matrix = {}
    for movieID,value in by_movie.collect():
        vector = [0 for _ in range(num_users)]
        for userID,rating in value:
            vector[int(userID)-1] = float(rating)
        matrix[int(movieID)] = vector
                                               
    return matrix

In [101]:
ratings_matrix = get_matrix(by_movie, num_users)

In [115]:
def calc_similarity(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

In [121]:
firstMovie = ratings_matrix[1]
secondMovie = ratings_matrix[12]

In [122]:
calc_similarity(firstMovie, secondMovie)

0.21967980853024835